![Pytorch](images/pytorch_logo.png)

# Regression in plain Pytorch
Let's rebuild our regression, but this time we use pytorch to handle our math!

In [ ]:
import torch

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Set seed
seed = 42
torch.manual_seed(seed);

## Load data

We are using the built-in sklearn dataset Boston House Prices.

Our goal is to predict the median price of a home in a given town from a number of features, such as Crime Rate, Property Tax Rate, amount of Industry etc.

It's generally a good idea to scale our data, so we use Sklearn's MinMax scaler to scale our values between 0 and 1

In [ ]:
# Load our dataset
boston = load_boston()
train_x, test_x, train_y, test_y = train_test_split(boston.data, boston.target, random_state=seed)
scaler = MinMaxScaler()

train_x = torch.tensor(scaler.fit_transform(train_x), dtype=torch.float)
test_x = torch.tensor(scaler.transform(test_x), dtype=torch.float)
train_y = torch.tensor(train_y, dtype=torch.float).view(-1, 1)
test_y = torch.tensor(test_y, dtype=torch.float).view(-1, 1)

## Setup parameters

We have some hyperparameters to set, as well as some numbers we need to know upfront.

`layer_size` --> We need to know how many input variables there are, so we can create an equivalent number of weights

`lr` --> Aka learning rate.
When we take a step in our gradient descent, we multiply by this factor, so we don't take too big or too large a step. 

`epochs` --> How many times should we keep stepping?

In [ ]:
layer_size = train_x.shape[1]
lr = 0.05
epochs = 700

## Initialize weights and bias

We need one weight to multiply each feature with - we are learning what these should be, so we start them as a random number. 

In [ ]:
# Initialize weights
w = torch.randn(layer_size, 1, requires_grad=True, dtype=torch.float)
b = torch.zeros(1, requires_grad=True, dtype=torch.float)

## Define Loss Function

Just like before, we want to use mean squared error to say how bad or good our line is

In [ ]:
# Define loss function
def mean_squared_error(y_hat, y):
    return ((y_hat - y) ** 2).mean()

In [ ]:
# Training loop
for epoch in range(epochs):
    # Forward pass
    pred = train_x @ w + b

    loss = mean_squared_error(pred, train_y)

    # Backpropagation
    loss.backward() # The magic bit!
    with torch.no_grad():
        w -= w.grad * lr
        b -= b.grad * lr
        w.grad.zero_() # Gotta reset the gradients to zero, so they don't accumulate
        b.grad.zero_()
        
        # Validate model
        val_pred = test_x @ w + b
        val_loss = mean_squared_error(val_pred, test_y)
        if epoch % 10 == 0:
            print(f"Epoch: {epoch} Train Loss: {loss.item()} Test Loss: {val_loss.item()}")